In [19]:
import rank_bm25
import sys
sys.path.append('../')  # Adjust the path as needed to go one directory up

from utils.common import log_step
from rank_bm25 import BM25Okapi


In [131]:

corpus = [
    "The rise of artificial intelligence is transforming industries.",
    "Climate change is a pressing global issue requiring urgent action.",
    "Advances in medical technology are improving patient outcomes.",
    "The history of the Roman Empire is rich with events and figures.",
    "Electric vehicles are gaining popularity due to environmental concerns.",
    "Space exploration has led to numerous technological advancements.",
    "Blockchain technology is revolutionizing digital transactions.",
    "Classical music has a profound influence on modern compositions.",
    "The Great Wall of China is a marvel of ancient engineering.",
    "Renewable energy sources are essential for sustainable development.",
    "The economic impacts of the COVID-19 pandemic are far-reaching.",
    "Artificial intelligence can enhance cybersecurity measures.",
    "The biodiversity of the Amazon rainforest is unparalleled.",
    "Quantum computing promises to solve complex problems faster.",
    "The Renaissance was a period of great cultural and scientific achievements.",
    "Social media platforms influence public opinion and behavior.",
    "Machine learning algorithms are improving data analysis.",
    "Ocean conservation efforts are crucial for marine life protection.",
    "The evolution of the internet has changed communication forever.",
    "The Mona Lisa is one of the most famous paintings in the world.",
    "Autonomous vehicles rely on sophisticated sensor technologies.",
    "The periodic table organizes chemical elements by their properties.",
    "Mental health awareness is important for societal well-being.",
    "Renewable energy technologies include solar and wind power.",
    "Artificial intelligence in healthcare can improve diagnostics.",
    "The Industrial Revolution marked a major turning point in history.",
    "The impact of deforestation on climate change is significant.",
    "The theory of relativity revolutionized physics.",
    "The importance of early childhood education cannot be overstated.",
]




In [129]:
corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(corpus)
query = "artificial intelligence in healthcare"
tokenized_query = query.split(" ")

arrScores = bm25.get_scores(tokenized_query)
score_dict = {i + 1: score for i, score in enumerate(arrScores)}
print(arrScores)
score_dict
print(score_dict)
sorted(score_dict.items(), key=lambda x: x[1], reverse=True)[:10]
# [(25, 1.9662271007766643), (12, 1.9103007731352675), (1, 1.8919125652468765), (15, 1.8846763798085642), (5, 1.8807901393580857), (9, 1.8692901488820168), (23, 1.860823296133419), (27, 1.844817805501623), (2, 1.8400733824099944), (7, 1.8324839261388206)]

[5.13520126 0.         1.79268402 0.         0.         0.
 0.         0.         0.         0.         0.         2.34520594
 0.         0.         0.         0.         0.         0.
 0.         1.41185175 0.         0.         0.         0.
 7.32294276 1.618098   0.         0.         0.        ]
{1: 5.135201264637504, 2: 0.0, 3: 1.7926840175569516, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 2.3452059366274427, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 1.4118517462055236, 21: 0.0, 22: 0.0, 23: 0.0, 24: 0.0, 25: 7.322942756228267, 26: 1.6180979994292037, 27: 0.0, 28: 0.0, 29: 0.0}


[(25, 7.322942756228267),
 (1, 5.135201264637504),
 (12, 2.3452059366274427),
 (3, 1.7926840175569516),
 (26, 1.6180979994292037),
 (20, 1.4118517462055236),
 (2, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0)]

In [134]:

bm25Off_instance = BM25Official2(corpus)
bm25Off_instance.search(q=query, top_k=10)

[(25, 7.322942756228267),
 (1, 5.135201264637504),
 (12, 2.3452059366274427),
 (3, 1.7926840175569516),
 (26, 1.6180979994292037),
 (20, 1.4118517462055236),
 (2, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0)]

In [5]:
bm25.get_top_n(tokenized_query, corpus, n=1)

['It is quite windy in London']

In [50]:
import pandas as pd
import sys
import logging
import itertools
from tqdm import tqdm

import math
import numpy as np
from statistics import mean
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from models.lexical_models import BM25Retriever

In [27]:
articles = pd.read_pickle('../../../../fr_lem_articles.pkl')
questions = pd.read_pickle('../../../../fr_lem_questions.pkl')
que = questions[:7]
dfQ_test = pd.read_csv("../../../../new_data/dutch/du_test.csv")
ground_truths = dfQ_test['article_ids'].apply(lambda x: list(map(int, x.split(',')))).tolist()
ground_true = ground_truths[:7]

In [135]:
corpus = [
    "The rise of artificial intelligence is transforming industries.",
    "Climate change is a pressing global issue requiring urgent action.",
    "Advances in medical technology are improving patient outcomes.",
    "The history of the Roman Empire is rich with events and figures.",
    "Electric vehicles are gaining popularity due to environmental concerns.",
    "Space exploration has led to numerous technological advancements.",
    "Blockchain technology is revolutionizing digital transactions.",
    "Classical music has a profound influence on modern compositions.",
    "The Great Wall of China is a marvel of ancient engineering.",
    "Renewable energy sources are essential for sustainable development.",
    "The economic impacts of the COVID-19 pandemic are far-reaching.",
    "Artificial intelligence can enhance cybersecurity measures.",
    "The biodiversity of the Amazon rainforest is unparalleled.",
    "Quantum computing promises to solve complex problems faster.",
    "The Renaissance was a period of great cultural and scientific achievements.",
    "Social media platforms influence public opinion and behavior.",
    "Machine learning algorithms are improving data analysis.",
    "Ocean conservation efforts are crucial for marine life protection.",
    "The evolution of the internet has changed communication forever.",
    "The Mona Lisa is one of the most famous paintings in the world.",
    "Autonomous vehicles rely on sophisticated sensor technologies.",
    "The periodic table organizes chemical elements by their properties.",
    "Mental health awareness is important for societal well-being.",
    "Renewable energy technologies include solar and wind power.",
    "Artificial intelligence in healthcare can improve diagnostics.",
    "The Industrial Revolution marked a major turning point in history.",
    "The impact of deforestation on climate change is significant.",
    "The theory of relativity revolutionized physics.",
    "The importance of early childhood education cannot be overstated."
]

# Create the BM25Official and TFIDFRetriever instances
bm25Off_instance = BM25Official2(retrieval_c=corpus)
tfidf_instance = TFIDFRetriever(corpus)
bm25_instance = BM25Retriever(corpus, k1=1.2, b=0.6)

# Example query
query = "artificial intelligence in healthcare"
top_k = 10

# Perform the search using BM25
bm25_results = bm25_instance.search(query, top_k)
print("BM25 Results:", bm25_results)

# Perform the search using BM25
bm25Off_results = bm25Off_instance.search(q=query, top_k=top_k)
print("BM25Off Results:", bm25Off_results)

# Perform the search using TFIDF
tfidf_results = tfidf_instance.search(query, top_k)
print("TFIDF Results:", tfidf_results)


BM25 Results: [(1, 2.2100072374434845), (25, 1.8699657915877381), (12, 0.9763119978044598), (2, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.0)]
BM25Off Results: [(25, 7.322942756228267), (1, 5.135201264637504), (12, 2.3452059366274427), (3, 1.7926840175569516), (26, 1.6180979994292037), (20, 1.4118517462055236), (2, 0.0), (4, 0.0), (5, 0.0), (6, 0.0)]
TFIDF Results: [(25, 2.1416813258827183), (1, 2.0217060088059684), (12, 0.8603380065709937), (3, 0.11997531707674983), (20, 0.11997531707674983), (26, 0.11997531707674983), (2, 0.0), (4, 0.0), (5, 0.0), (6, 0.0)]


In [43]:
retrieverTFIDF.search_all(queries, top_k=top_k)


Searching queries: 100%|██████████| 2/2 [00:00<00:00, 2002.05it/s]

Just ran 'search_all' function. Took: 0:00:00.002294


[[5, 8, 17, 19, 20], [1, 2, 3, 4, 5]]

In [44]:
retrieverBM25.search_all(queries, top_k=top_k)

Searching queries: 100%|██████████| 2/2 [00:00<?, ?it/s]

Just ran 'search_all' function. Took: 0:00:00.002547


[[20, 17, 27, 14, 13], [17, 12, 48, 21, 45]]

In [133]:
class BM25Official2:
    def __init__(self, retrieval_c):
        corpus = [doc.split(" ") for doc in retrieval_c]
        self.bm25 = BM25Okapi(corpus)
        
    def __repr__(self):
        return f"{self.__class__.__name__}".lower()

    @log_step
    def search_all(self, queries, top_k):
        results = list()
        for q in tqdm(queries, desc='Searching queries'):
            results.append([doc_id for doc_id,_ in self.search(q, top_k)])
        return results

    def search(self, q, top_k):
        tokenized_query = q.split(" ")
        scores = self.bm25.get_scores(tokenized_query)
        results = {i + 1: score for i, score in enumerate(scores)}
        return sorted(results.items(), key=lambda x: x[1], reverse=True)[:top_k]


class TFIDFRetriever:
    def __init__(self, retrieval_corpus):
        self.retrieval_corpus = retrieval_corpus
        self.N = len(retrieval_corpus)
        self.vocab = self._build_vocabulary()
        self.idfs = self._compute_idfs()

    def __repr__(self):
        return f"{self.__class__.__name__}".lower()

    @log_step
    def search_all(self, queries, top_k):
        results = list()
        for q in tqdm(queries, desc='Searching queries'):
            results.append([doc_id for doc_id,_ in self.search(q, top_k)])
        return results

    #So returns tuples with (art_id, score) sorted high to low
    def search(self, q, top_k):
        results = dict()
        for i, doc in enumerate(self.retrieval_corpus):
            results[i+1] = self.score(q, doc) #NB: '+1' because doc_ids in BSARD start at 1.
        return sorted(results.items(), key=lambda x: x[1], reverse=True)[:top_k]

    def score(self, q, d):
        score = 0.0
        for t in q.split():
            score += self._compute_tfidf(t, d)
        return score

    #Returns set of vocabulary
    def _build_vocabulary(self):
        return sorted(set(itertools.chain.from_iterable([doc.lower().split() for doc in self.retrieval_corpus])))

    def _compute_idfs(self):
        idfs = dict.fromkeys(self.vocab, 0)
        for word,_ in idfs.items():
            idfs[word] = self._compute_idf(word)
        return idfs

    def _compute_idf(self, t):
        df = sum([1 if t in doc else 0 for doc in self.retrieval_corpus])
        return math.log10(self.N / (df + 1))

    def _compute_tf(self, t, d):
        return d.split().count(t)

    def _compute_tfidf(self, t, d):
        tf = self._compute_tf(t, d)
        idf = self.idfs[t] if t in self.idfs else math.log10(self.N)
        return tf * idf